A continuación verá una función preestablecida, donde:

- models - lista de objetos del algoritmo base

- meta_alg - meta-algoritmo

data_train, target_train, data_test, target_test - atributos de entrenamiento y prueba y variables objetivo

- test_size - tamaño de los datos de prueba para la mezcla en el intervalo (0, 1)

In [7]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [12]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

def stacking(models, meta_alg, data_train, targets_train, data_test, targets_test=None, random_state=None, test_size=None, cv=5):
    if test_size is None:
        pass
    
    elif 0 < test_size < 1:
        # División en datos de entrenamiento y validación
        train_data, val_data, train_targets, val_targets = train_test_split(data_train, targets_train, test_size=test_size, random_state=random_state)
        
        # Definición de la matriz meta_mtrx
        meta_mtrx = np.zeros((val_data.shape[0], len(models)))
        
        # Bucle para llenar la matriz meta_mtrx
        for i, model in enumerate(models):
            # Entrenamiento de un modelo básico sobre datos de entrenamiento
            model.fit(train_data, train_targets)
            
            # Llenar una columna de matriz con predicciones basadas en datos de validación
            meta_mtrx[:, i] = model.predict(val_data)
        
        # Metaalgoritmo de entrenamiento en la matriz meta_mtrx
        meta_alg.fit(meta_mtrx, val_targets)
        
        # Definición de la matriz meta_mtrx_test
        meta_mtrx_test = np.zeros((data_test.shape[0], len(models)))
        
        # Bucle para llenar la matriz meta_mtrx_test
        for i, model in enumerate(models):
            # Llenar la columna de la matriz con predicciones sobre los datos de prueba
            meta_mtrx_test[:, i] = model.predict(data_test)
        
        # Predicciones de metaalgoritmo para valores meta_mtrx_test
        meta_predictions = meta_alg.predict(meta_mtrx_test)
        
        # Comprobando objetivos_prueba y generando precisión_puntuación
        if targets_test is not None:
            accuracy = accuracy_score(targets_test, meta_predictions)
            print(f"Accuracy on Test Data: {accuracy}")
    
    else:
        raise ValueError("test_size must be between 0 and 1")

# Ejemplo de uso de la función
# usaremos el conjunto de datos de Iris como ejemplo
iris = load_iris()
data_train, data_test, targets_train, targets_test = train_test_split(iris.data, iris.target, test_size=0.3, random_state=42)

# Definición de algoritmos básicos
base_models = [
    RandomForestClassifier(n_estimators=50, random_state=42),
    GradientBoostingClassifier(n_estimators=50, random_state=42),
    DecisionTreeClassifier(random_state=42),
    SVC(random_state=17)

]

# Definición de algoritmos básicos
meta_model = LogisticRegression(random_state=42, multi_class='ovr')

# Probando la función de apuesta en tres variantes diferentes
try:
    # Opción 1: lanzar una excepción
    stacking(base_models, meta_model, data_train, targets_train, data_test, targets_test=None, test_size=None, cv=5)
except ValueError as e:
    print(f"Opción 1: {e}")

# Opción 2: configurar test_size=0.3
stacking(base_models, meta_model, data_train, targets_train, data_test, targets_test, test_size=0.3, cv=5)

# Opción 3: test_size=Ninguno
stacking(base_models, meta_model, data_train, targets_train, data_test, targets_test, test_size=None, cv=5)


Accuracy on Test Data: 1.0
